In [ ]:
import folium
import DownloadData as DD
import DataSorting as DS
import json
import geojson
from urllib.request import urlopen
from geojson_utils import centroid

In [ ]:
# récupération des contours régionaux
with urlopen('https://raw.githubusercontent.com/gregoiredavid/france-geojson/master/regions-version-simplifiee.geojson') as f:
    regions = geojson.load(f)

In [ ]:
# récupération des données sur les stations-service
DD.get_ten_Mins_Data()
DD.get_dailyData()

# on récupère le J-5 pour observer la variation 
chaine = DD.get_chaine()

# calcul des prix moyen pour les régions/départements/cantons
reg_prices = DS.DictRegions("PrixCarburants_instantane.xml")
reg_prices_B = DS.DictRegions("PrixCarburants_quotidien_" + chaine + ".xml")

# calcul des variations
variations_reg = {key: reg_prices[key] - reg_prices_B.get(key, 0) for key in reg_prices.keys()}

In [ ]:
# tendance régionale
average_reg = folium.Map(location=[46.232192999999995, 2.209666999999996], zoom_start=5)

folium.Choropleth(
    geo_data=regions,
    name="choropleth",
    data=reg_prices,
    key_on="feature.properties.nom",
    fill_color="YlOrRd",
    nan_fill_color="White", #Use white color if there is no data available for the departements
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name="Diesel prices in France regions", #title of the legend
    highlight=True,
    line_color='black'
).add_to(average_reg)

folium.LayerControl().add_to(average_reg)

average_reg

In [ ]:
# ajout des flèches de variations
for i in range (0, len(regions["features"])):
    if regions["features"][i].geometry.type == 'Polygon':
        box_str = str(regions["features"][i].geometry)
        box = json.loads(box_str)
        center = centroid(box)
        loc = [center['coordinates'][1], center['coordinates'][0]]
        nom_r = regions["features"][i]["properties"]["nom"]
        if nom_r in variations_reg:
            pop = f'{nom_r}</br>{round(variations_reg[nom_r], 4)}'
            if variations_reg[nom_r] < 0:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/down.png',icon_size=(12,12))).add_to(average_reg)
            elif 0 < variations_reg[nom_r] <= max(variations_reg.values())/1.5:
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_1.png',icon_size=(12,12))).add_to(average_reg)
            elif max(variations_reg.values())/1.5 < variations_reg[nom_r] <= max(variations_reg.values()):
                folium.Marker(location = loc,
                                  popup = pop,
                                  icon = folium.features.CustomIcon('icon/up_2.png',icon_size=(12,12))).add_to(average_reg)
            else:
                folium.Marker(location = loc,
                                  popup = nom_r,
                                  icon = folium.features.CustomIcon('icon/equal.png',icon_size=(16,16))).add_to(average_reg)
                
average_reg